# Sistema de asistencia basado en el reconocimiento facial

## Introducción

Este proyecto muestra un Sistema de asistencia basado en el reconocimiento facial desarrollado en Python. Los sistemas tradicionales de control de asistencia suelen ser manuales, propensos a errores y lentos. Aprovechando el reconocimiento facial, podemos automatizar el proceso, creando una solución más eficiente, precisa y rentable. Esto es especialmente beneficioso para instituciones como escuelas y empresas, donde el control de asistencia de un gran número de personas es una tarea diaria. La automatización elimina los errores humanos, evita las entradas fraudulentas (como la asistencia por poder) y libera un tiempo valioso para actividades más productivas.


## Antecedentes y motivación

Los sistemas manuales de control de asistencia adolecen de varios inconvenientes, como la posibilidad de cometer errores, el posible robo de tiempo o falsificación de registros asistenciales, y la carga administrativa que supone compilar manualmente los registros. Grandes empresas tecnológicas como Amazon, Microsoft y Face++ ya han demostrado el poder del reconocimiento facial para la seguridad, el control de acceso y la identificación de usuarios.

La motivación de este proyecto es aplicar conceptos fundamentales de la visión por ordenador y el aprendizaje automático para crear una aplicación práctica en el mundo real. El objetivo es crear un prototipo accesible y de bajo coste que pueda implementarse utilizando una cámara web estándar y bibliotecas Python de código abierto, convirtiéndolo en una opción viable para organizaciones con presupuestos limitados.


## Desarrollo del proyecto

### Herramientas utilizadas

* Lenguaje de programación:Python
* **Bibliotecas:**
    * `NumPy`: Para operaciones numéricas y manejo de datos de imagen como matrices.
    * `OpenCV`: Para procesamiento y manejo de imágenes.
    * `face_recognition`: Una librería sencilla y potente para detectar, reconocer y manipular caras.
    * `cmake`: Herramienta que configura y gestiona la compilación de bibliotecas como `dlib`.
    * `dlib`: Biblioteca que proporciona algoritmos para la deteción y codificación de rostros, fundamentales para el reconocimiento facial.
* **Hardware:** Ordenador portátil o de sobremesa con webcam (para captura de imágenes).
* **Entorno de desarrollo:** Cursor

### Paso 1: Instalar e importar las librerías necesarias

En primer lugar, tenemos que instalar las bibliotecas necesarias. `dlib` es un prerrequisito para `face_recognition`. También instalaremos `cmake` para asegurarnos de que `dlib` compila correctamente.

Adicionalmente, en caso de errores al tratar de instalar CMake, descargalo desde su sitio web oficial: https://cmake.org/download/. Abrí el instalador, marcá la opción "Add CMake to system PATH for all users", instalá, reiniciá la terminal y corré esta línea de código:


`cmake --version`

Deberías recibir un mensaje con la versión instalada de dicho sistema.

Por otro lado, en caso de tener errores en la instalación de dlib, descargá Visual Studio 2022, en su versión Community, desde https://visualstudio.microsoft.com/es/downloads/. Luego, en el instalador, seleccioná "Desarrollo para el escritorio con C++" y continuá con la instalación. Finalmente, reiniciá la terminal e intentá instalar las dependencias nuevamente.

In [ ]:
!pip install requirements.txt

Ahora, vamos a importar las bibliotecas que vamos a utilizar en todo el proyecto.

In [1]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime

### Paso 2: Cargar imágenes de personas conocidas

Necesitamos una base de datos de caras conocidas con las que comparar. Para este notebook, crearemos un directorio llamado `known_faces` y cargaremos imágenes en él. Cada archivo de imagen debe llevar el nombre de la persona que aparece en la foto (por ejemplo, `elon_musk.jpg`, `bill_gates.png`).

In [ ]:
# --- PASO 2: ENRUTADOR A LOS ROSTROS CONOCIDOS ---

# Ruta al directorio con las imágenes de las personas conocidas.
known_faces_dir = 'C:/Users/Ruta_local'

### Reemplazar la ruta actual por la nueva ruta local



### Paso 3: Codificar 'Known Faces'

El núcleo del reconocimiento facial es convertir una cara en una representación matemática, llamada «codificación». Se trata de un vector de 128 números que es único para cada cara. Ahora procesaremos cada imagen cargada, detectaremos la cara y generaremos su codificación. Almacenaremos estas codificaciones junto con el nombre de la persona.

In [3]:
# --- PASO 3: CODIFICAR LOS ROSTROS CONOCIDOS ---
# 
# # Listas para almacenar las codificaciones de los rostros conocidos y sus nombres.
known_face_encodings = []
known_face_names = []

print("Procesando rostros conocidos...")

# Recorrer cada archivo en el directorio de rostros conocidos.
for filename in os.listdir(known_faces_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Cargar el archivo de imagen.
        image_path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(image_path)

        # Obtener la codificación del rostro (se asume una cara por imagen).
        face_encodings = face_recognition.face_encodings(image)

        if face_encodings:
            encoding = face_encodings[0]
            # Agregar la codificación y el nombre (sin la extensión del archivo) a las listas.
            known_face_encodings.append(encoding)
            known_face_names.append(os.path.splitext(filename)[0])

print(f"Se procesaron {len(known_face_names)} rostros conocidos.")
print("Nombres conocidos:", known_face_names)

Procesando rostros conocidos...
Se procesaron 6 rostros conocidos.
Nombres conocidos: ['Adrian_DIAZ', 'Ana_DEARMAS', 'Daiana_FRETE', 'Hanna_DIAZ', 'Michael_MANDO', 'Nataly_PEREZ']


### PASO 4: REGISTRAR LOS ROSTROS CONOCIDOS COMO ASISTENCIAS


Se registra la asistencia de una persona en el archivo `attendance.csv`, guardando su nombre y la hora exacta en que fue detectada. Antes de agregar una nueva entrada, verifica si esa persona ya fue registrada en la sesión actual, evitando duplicados. Usa el modo de archivo 'a+' para permitir lectura y escritura, y para crear el archivo si no existe.

In [4]:
# --- PASO 4: FUNCIÓN PARA MARCAR LA ASISTENCIA ---

def mark_attendance(name):
    """
    Registra el nombre y la hora en el archivo attendance.csv.
    Evita registrar a la misma persona más de una vez por sesión.
    """
    # Usar 'a+' para abrir el archivo en modo de anexo (lo crea si no existe).
    with open('attendance.csv', 'a+') as f:
        # Mover el cursor al inicio para leer las entradas existentes.
        f.seek(0)
        my_data_list = f.readlines()
        name_list = []
        for line in my_data_list:
            entry = line.split(',')
            name_list.append(entry[0])

        # Comprobar si el nombre no ha sido ya registrado.
        if name not in name_list:
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dt_string}')
            print(f"Asistencia marcada para {name}")

### Paso 5: Capturar una Imagen para la Asistencia

Ahora es el momento de pasar lista. En una aplicación real, esto implicaría capturar un fotograma en directo de una cámara web.

In [ ]:
# --- PASO 5: CAPTURAR VIDEO Y REALIZAR EL RECONOCIMIENTO FACIAL ---

# Iniciar la captura de video desde la cámara web (el índice 0 suele ser la cámara por defecto).
video_capture = cv2.VideoCapture(0)

print("\nIniciando cámara. Presiona 'q' para salir.")

while True:
    # Capturar un solo fotograma de video.
    ret, frame = video_capture.read()
    if not ret:
        print("Error: No se pudo capturar el fotograma.")
        break

    # Redimensionar el fotograma para un procesamiento más rápido (opcional).
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convertir la imagen de BGR (OpenCV) a RGB (face_recognition).
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Encontrar todas las ubicaciones y codificaciones de rostros en el fotograma actual.
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    # Recorrer cada rostro encontrado en el fotograma.
    for face_encoding, face_loc in zip(face_encodings, face_locations):
        # Comparar el rostro actual con la lista de rostros conocidos.
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Desconocido"  # Nombre por defecto si no hay coincidencia.

        # Encontrar la mejor coincidencia usando la distancia euclidiana.
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            mark_attendance(name)

        # Dibujar un cuadro alrededor del rostro y mostrar el nombre.
        # Escalar las ubicaciones del rostro al tamaño original del fotograma.
        top, right, bottom, left = face_loc
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Dibujar el rectángulo y la etiqueta con el nombre.
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Mostrar la imagen resultante.
    cv2.imshow('Reconocimiento Facial - Asistencia (Presiona "q" para salir)', frame)

    # Salir del bucle si se presiona la tecla 'q'.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



Iniciando cámara. Presiona 'q' para salir.
Asistencia marcada para Adrian_DIAZ
Asistencia marcada para Daiana_FRETE
Asistencia marcada para Hanna_DIAZ
Asistencia marcada para Nataly_PEREZ
Asistencia marcada para Ana_DEARMAS


: 

### Paso 6: Comparar las caras capturadas y marcar la asistencia

Este es el paso final. Haremos lo siguiente
1.  Cargar la imagen de asistencia cargada.
2.  Encontrar todas las caras y sus codificaciones en esta imagen.
3.  Comparar cada cara encontrada con nuestra base de datos de codificaciones de caras conocidas.
4.  Si se encuentra una coincidencia, identificaremos a la persona.
5.  Dibujar un recuadro alrededor de la cara y etiquetarlo con el nombre.
6.  Registrar la asistencia en un fichero CSV.

In [ ]:
# Function to mark attendance
def mark_attendance(name):
    # Use 'a+' to create the file if it doesn't exist and append to it
    with open('attendance.csv', 'a+') as f:
        # Move cursor to the start to read existing entries
        f.seek(0)
        my_data_list = f.readlines()
        name_list = []
        for line in my_data_list:
            entry = line.split(',')
            name_list.append(entry[0])

        # Check if the name is not already in the list for today
        if name not in name_list:
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dt_string}')
            print(f"Attendance marked for {name}")

# Load the attendance image
img = cv2.imread(attendance_image_name)
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Find face locations and encodings in the current frame
face_locations = face_recognition.face_locations(rgb_img)
face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

print(f"Found {len(face_locations)} face(s) in the attendance image.")

# Loop through each face found in the attendance image
for face_encoding, face_loc in zip(face_encodings, face_locations):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
    name = "Unknown" # Default name if no match is found

    # Use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]
        mark_attendance(name)

    # Draw a box around the face and display the name
    top, right, bottom, left = face_loc
    cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
    cv2.rectangle(img, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(img, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

# Display the resulting image
print("\nDisplaying result:")
cv2_imshow(img)


### Paso 7: Comprobar el registro de asistencia

Por último, vamos a ver el contenido de nuestro archivo `attendance.csv`. Este archivo registra el nombre de la persona reconocida y la fecha y hora en que se marcó su asistencia. Cada persona sólo se marca una vez por sesión.

In [ ]:
# --- PASO 7: LIBERAR RECURSOS ---

# Liberar el manejador de la cámara web.
video_capture.release()
# Cerrar todas las ventanas de OpenCV.
cv2.destroyAllWindows()

print("\n--- Registro de Asistencia Final ---")
try:
    with open('attendance.csv', 'r') as f:
        print(f.read())
except FileNotFoundError:
    print("El archivo de asistencia no se creó. Nadie fue reconocido.")